In [1]:
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import auc
import pandas as pd
from tqdm import tqdm
import os
from sklearn.metrics import confusion_matrix

In [2]:
def find_best_percent(result, granularity_all=1000):
    max_percent = 0
    best_n = 1
    for n in tqdm(range(1, 100)):
        head_n = n / granularity_all
        data_length = max(round(len(result) * head_n), 1)
        count_dist = count_entries(result.loc[:data_length - 1], 'label')
        try:
            percent = count_dist['1'] / (count_dist['0'] + count_dist['1'])
        except KeyError:
            print("can't find n%,take 1%")
            percent = 0.01
        if percent > max_percent:
            max_percent = percent
            best_n = n
    return best_n, max_percent, granularity_all

def count_entries(df, col_name):
    count_dist = {'0': 0, '1': 0}
    col = df[col_name]
    for entry in col:
        if str(int(entry)) in count_dist.keys():
            count_dist[str(int(entry))] = count_dist[str(int(entry))] + 1
        else:
            count_dist[str(int(entry))] = 1
    return count_dist

def find_best_result(threshold_n, result, dataframe_std):
    best_result, best_h, best_re, best_fa, best_f1, best_precision = None, 0, 0, 0, 0, 0
    best_auroc = 0
    for h in tqdm(range(50, 1000, 50)):
        train_result = charge_to_car(threshold_n, result, head_n=h)
        f1, recall, false_rate, precision, accuracy, auroc = evaluation(dataframe_std, train_result)
        print("find_best_result", auroc)
        if auroc >= best_auroc:
            best_f1 = f1
            best_h = h
            best_re = recall
            best_fa = false_rate
            best_result = train_result
            best_auroc = auroc
    return best_result, best_h, best_re, best_fa, best_f1, best_auroc

def charge_to_car(threshold_n, rec_result, head_n=92):
    gran = 1000
    result = []
    for grp in rec_result.groupby('car'):
        temp = grp[1].values[:, -1].astype(float)
        idx = max(round(head_n / gran * len(temp)), 1)
        error = np.mean(temp[:idx])
        result.append([grp[0], int(error > threshold_n), error, threshold_n])
    return pd.DataFrame(result, columns=['car', 'predict', 'error', 'threshold_n'])

def evaluation(dataframe_std, dataframe):
    
    # calculate auroc
#     print(dataframe) # error car
    _label = []
    for each_car in dataframe['car']:
        if int(each_car) in ind_car_num_list:
            _label.append(0)
        if int(each_car) in ood_car_num_list:
            _label.append(1)
    
    fpr, tpr, thresholds = metrics.roc_curve(_label, list(dataframe['error']), pos_label=1)
    auroc = auc(fpr, tpr)
    
    
    data = pd.merge(dataframe_std, dataframe, on='car')
    cm = confusion_matrix(data['label'].astype(int), data['predict'].astype(int))
    tn = cm[0, 0]
    fp = cm[0, 1]
    fn = cm[1, 0]
    tp = cm[1, 1]
    precision = tp / (tp + fp) if tp + fp != 0 else 0
    recall = tp / (tp + fn) if tp + fn != 0 else 0
    false_rate = fp / (tn + fp) if tn + fp != 0 else 0
    accuracy = (tp + tn) / (tp + tn + fp + fn) if tp + tn + fp + fn != 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall != 0 else 0
    return f1, recall, false_rate, precision, accuracy, auroc

# change to your corresponding dir

In [ ]:
# ! ls ../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/

In [ ]:
batterydatasetall
df0_actual = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134410fold_0/actual.txt.npy')
df0_car_charge_segment = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134410fold_0/car_charge_segment.txt.npy')
df0_car_head = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134410fold_0/car_head.txt.npy')
df0_car_nums = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134410fold_0/car_nums.txt.npy')
df0_preds = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134410fold_0/preds.txt.npy')
df0_recons = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134410fold_0/recons.txt.npy')

df1_actual = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134418fold_1/actual.txt.npy')
df1_car_charge_segment = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134418fold_1/car_charge_segment.txt.npy')
df1_car_head = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134418fold_1/car_head.txt.npy')
df1_car_nums = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134418fold_1/car_nums.txt.npy')
df1_preds = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134418fold_1/preds.txt.npy')
df1_recons = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134418fold_1/recons.txt.npy')

df2_actual = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134427fold_2/actual.txt.npy')
df2_car_charge_segment = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134427fold_2/car_charge_segment.txt.npy')
df2_car_head = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134427fold_2/car_head.txt.npy')
df2_car_nums = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134427fold_2/car_nums.txt.npy')
df2_preds = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134427fold_2/preds.txt.npy')
df2_recons = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134427fold_2/recons.txt.npy')

df3_actual = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134441fold_3/actual.txt.npy')
df3_car_charge_segment = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134441fold_3/car_charge_segment.txt.npy')
df3_car_head = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134441fold_3/car_head.txt.npy')
df3_car_nums = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134441fold_3/car_nums.txt.npy')
df3_preds = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134441fold_3/preds.txt.npy')
df3_recons = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134441fold_3/recons.txt.npy')

df4_actual = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134453fold_4/actual.txt.npy')
df4_car_charge_segment = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134453fold_4/car_charge_segment.txt.npy')
df4_car_head = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134453fold_4/car_head.txt.npy')
df4_car_nums = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134453fold_4/car_nums.txt.npy')
df4_preds = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134453fold_4/preds.txt.npy')
df4_recons = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND123/20052023_134453fold_4/recons.txt.npy')

# batterydataset1
# df0_actual = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_194042fold_0/actual.txt.npy')
# df0_car_charge_segment = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_194042fold_0/car_charge_segment.txt.npy')
# df0_car_head = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_194042fold_0/car_head.txt.npy')
# df0_car_nums = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_194042fold_0/car_nums.txt.npy')
# df0_preds = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_194042fold_0/preds.txt.npy')
# df0_recons = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_194042fold_0/recons.txt.npy')

# df1_actual = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_134628fold_1/actual.txt.npy')
# df1_car_charge_segment = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_134628fold_1/car_charge_segment.txt.npy')
# df1_car_head = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_134628fold_1/car_head.txt.npy')
# df1_car_nums = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_134628fold_1/car_nums.txt.npy')
# df1_preds = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_134628fold_1/preds.txt.npy')
# df1_recons = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_134628fold_1/recons.txt.npy')

# df2_actual = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_134630fold_2/actual.txt.npy')
# df2_car_charge_segment = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_134630fold_2/car_charge_segment.txt.npy')
# df2_car_head = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_134630fold_2/car_head.txt.npy')
# df2_car_nums = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_134630fold_2/car_nums.txt.npy')
# df2_preds = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_134630fold_2/preds.txt.npy')
# df2_recons = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_134630fold_2/recons.txt.npy')

# df3_actual = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_134651fold_3/actual.txt.npy')
# df3_car_charge_segment = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_134651fold_3/car_charge_segment.txt.npy')
# df3_car_head = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_134651fold_3/car_head.txt.npy')
# df3_car_nums = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_134651fold_3/car_nums.txt.npy')
# df3_preds = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_134651fold_3/preds.txt.npy')
# df3_recons = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_134651fold_3/recons.txt.npy')

# df4_actual = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_194051fold_4/actual.txt.npy')
# df4_car_charge_segment = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_194051fold_4/car_charge_segment.txt.npy')
# df4_car_head = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_194051fold_4/car_head.txt.npy')
# df4_car_nums = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_194051fold_4/car_nums.txt.npy')
# df4_preds = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_194051fold_4/preds.txt.npy')
# df4_recons = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND1/20052023_194051fold_4/recons.txt.npy')

# batterydataset3
# df0_actual = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_195838fold_0/actual.txt.npy')
# df0_car_charge_segment = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_195838fold_0/car_charge_segment.txt.npy')
# df0_car_head = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_195838fold_0/car_head.txt.npy')
# df0_car_nums = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_195838fold_0/car_nums.txt.npy')
# df0_preds = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_195838fold_0/preds.txt.npy')
# df0_recons = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_195838fold_0/recons.txt.npy')

# df1_actual = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_195845fold_1/actual.txt.npy')
# df1_car_charge_segment = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_195845fold_1/car_charge_segment.txt.npy')
# df1_car_head = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_195845fold_1/car_head.txt.npy')
# df1_car_nums = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_195845fold_1/car_nums.txt.npy')
# df1_preds = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_195845fold_1/preds.txt.npy')
# df1_recons = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_195845fold_1/recons.txt.npy')

# df2_actual = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_200620fold_2/actual.txt.npy')
# df2_car_charge_segment = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_200620fold_2/car_charge_segment.txt.npy')
# df2_car_head = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_200620fold_2/car_head.txt.npy')
# df2_car_nums = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_200620fold_2/car_nums.txt.npy')
# df2_preds = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_200620fold_2/preds.txt.npy')
# df2_recons = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_200620fold_2/recons.txt.npy')

# df3_actual = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_200644fold_3/actual.txt.npy')
# df3_car_charge_segment = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_200644fold_3/car_charge_segment.txt.npy')
# df3_car_head = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_200644fold_3/car_head.txt.npy')
# df3_car_nums = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_200644fold_3/car_nums.txt.npy')
# df3_preds = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_200644fold_3/preds.txt.npy')
# df3_recons = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_200644fold_3/recons.txt.npy')

# df4_actual = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_200651fold_4/actual.txt.npy')
# df4_car_charge_segment = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_200651fold_4/car_charge_segment.txt.npy')
# df4_car_head = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_200651fold_4/car_head.txt.npy')
# df4_car_nums = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_200651fold_4/car_nums.txt.npy')
# df4_preds = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_200651fold_4/preds.txt.npy')
# df4_recons = np.load('../mtad-gat-pytorch-modified/output/BATTERY_BRAND3/20052023_200651fold_4/recons.txt.npy')

In [ ]:
AUC_list = []
gamma = 1

for fold_num in range(5):
#     fold_num = 3

    df_actual = eval('df'+str(fold_num)+'_actual')
    df_car_charge_segment = eval('df'+str(fold_num)+'_car_charge_segment')
    df_car_head = eval('df'+str(fold_num)+'_car_head')
    df_car_nums = eval('df'+str(fold_num)+'_car_nums')
    df_preds = eval('df'+str(fold_num)+'_preds')
    df_recons = eval('df'+str(fold_num)+'_recons')
    
    print(df_actual.shape)
    print(df_car_charge_segment.shape)
    print(df_car_head.shape)
    print(df_car_nums.shape)
    print(df_preds.shape)
    print(df_recons.shape)
    
    
    anomaly_scores = np.zeros_like(df_actual)
    for i in range(df_preds.shape[1]):
        a_score = np.sqrt((df_preds[:, i] - df_actual[:, i]) ** 2) + gamma * np.sqrt((df_recons[:, i] - df_actual[:, i]) ** 2)
        anomaly_scores[:, i] = a_score
    anomaly_scores = np.mean(anomaly_scores, 1)
    print(anomaly_scores.shape)
    
#     data = np.array(df)[:, 1:]
#     print(data.shape)
#     data = np.vstack((data[:,1], data[:,0], data[:,2])).T
#     print(data[:5,])
    
    

    # replace the following two car number lists
    ind_ood_car_dict = np.load('../five_fold_utils/ind_odd_dict.npz.npy', allow_pickle=True).item()
#     ind_ood_car_dict = np.load('../five_fold_utils/ind_odd_dict1.npz.npy', allow_pickle=True).item()
#     ind_ood_car_dict = np.load('../five_fold_utils/ind_odd_dict3.npz.npy', allow_pickle=True).item()
    ind_car_num_list = ind_ood_car_dict['ind_sorted']
    ood_car_num_list = ind_ood_car_dict['ood_sorted']
    
    label = np.zeros_like(df_car_nums)
    for each in df_car_nums:
        if each in ind_car_num_list:
            label[each] = 0
        else:
            label[each] = 1
    print(df_car_nums.shape, label.shape, anomaly_scores.shape)
    
    data = np.vstack((label, df_car_nums, anomaly_scores)).T

    
    # car_number = ind_car_num_list[int(fold_num * 175 / 5):int((fold_num + 1) * 175 / 5)] + ood_car_num_list

    validation_car_number = ind_car_num_list[:int(fold_num * len(ind_car_num_list) / 5)] + ind_car_num_list[int((fold_num + 1) * len(ind_car_num_list) / 5):] + ood_car_num_list[int(fold_num * len(ood_car_num_list) / 5):int((fold_num + 1) * len(ood_car_num_list) / 5)]
    test_car_number = ind_car_num_list[int(fold_num * len(ind_car_num_list) / 5):int((fold_num + 1) * len(ind_car_num_list) / 5)] + ood_car_num_list[:int(fold_num * len(ood_car_num_list) / 5)] + ood_car_num_list[int((fold_num + 1) * len(ood_car_num_list) / 5):]


    validation_data = []
    test_data = []

    for each_line in data:
        if each_line[1] in validation_car_number:
            validation_data.append(each_line)
        else:
            test_data.append(each_line)

    validation_data = np.array(validation_data)
    test_data = np.array(test_data)
    print(validation_data.shape, test_data.shape)

    rec_sorted_index = np.argsort(-validation_data[:, 2].astype(float))
    res = [validation_data[i][[1, 0, 2]] for i in rec_sorted_index]
    result = pd.DataFrame(res, columns=['car', 'label', 'rec_error'])
    result['car'] = result['car'].astype("int").astype("str")
    # result.head()

    best_n, max_percent, granularity = find_best_percent(result, granularity_all=1000)
    head_n = best_n / granularity
    data_length = round(len(result) * head_n)
    threshold_n = result['rec_error'].values[data_length - 1].astype(float)

    print("threshold_n", threshold_n)
    dataframe_std1 = pd.read_csv(os.path.join('../../../battery_dataset1/label', "label.csv"), dtype=object)
    dataframe_std2 = pd.read_csv(os.path.join('../../../battery_dataset2/label', "label.csv"), dtype=object)
    dataframe_std3 = pd.read_csv(os.path.join('../../../battery_dataset3/label', "label.csv"), dtype=object)
#     dataframe_std = pd.concat([dataframe_std1], axis=0,ignore_index=True)
#     dataframe_std = pd.concat([dataframe_std3], axis=0,ignore_index=True)
    dataframe_std = pd.concat([dataframe_std1, dataframe_std2, dataframe_std3], axis=0,ignore_index=True)
#     assert max(dataframe_std['car'].astype("int"))>300
#     assert 201 in dataframe_std['car'].astype("int")
    
    best_result, best_h, best_re, best_fa, best_f1, best_auroc = find_best_result(threshold_n, result, dataframe_std)
    # best_result.to_csv(os.path.join(self.args.result_path, "train_res.csv")),
    if dataframe_std.shape[0] == best_result.shape[0]:
        pass
    else:
        print('dataframe_std is ', dataframe_std.shape[0], '&&   dataframe is ', best_result.shape[0])

    rec_sorted_index = np.argsort(-test_data[:, 2].astype(float))
    res = [test_data[i][[1, 0, 2]] for i in rec_sorted_index]
    result = pd.DataFrame(res, columns=['car', 'label', 'rec_error'])
    result['car'] = result['car'].astype("int").astype("str")
    # result.to_csv(os.path.join(self.args.result_path, "test_segment_scores.csv"))
    test_result = charge_to_car(threshold_n, result, head_n=best_h)

    _score = list(test_result['error'])
    _label = []
    for each_car in test_result['car']:
        if int(each_car) in ind_car_num_list:
            _label.append(0)
        if int(each_car) in ood_car_num_list:
            _label.append(1)

    fpr, tpr, thresholds = metrics.roc_curve(_label, _score, pos_label=1)
    plt.plot(fpr, tpr)
    plt.show()
    AUC = auc(fpr, tpr)
    print('AUC', AUC)
    AUC_list.append(AUC)
    
#     assert 1==0
    
    os.makedirs("../mtad-gat-pytorch-modified/robust_battery_brand123/", exist_ok=True)
    np.save('../mtad-gat-pytorch-modified/robust_battery_brand123/robust_score_fold%d' % fold_num, _score)
    np.save('../mtad-gat-pytorch-modified/robust_battery_brand123/robust_label_fold%d' % fold_num, _label)

In [ ]:
print(AUC_list)
print(np.mean(AUC_list))
print(np.std(AUC_list))
AUC_list_round = ['%.4f' % each for each in AUC_list]
print(AUC_list_round)
